# Setup

In [44]:
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib
import re

In [8]:
# total document files in our corpus
len(os.listdir('english-corpora'))

8635

In [15]:
# all of them are .txt files
all('.txt' in t for t in os.listdir('english-corpora'))

True

In [16]:
os.listdir('english-corpora')[:5]

['D00521.txt', 'P_1709.txt', 'D00920.txt', 'T00143.txt', 'P03064.txt']

In [ ]:
# TODO: how to remove HTML info from these pages
# TODO: learn how to use porter's stemmer
# TODO: learn how to do tokenization in python
# TODO: how to clean text files: remove non-ascii and what else?; and when removing non-ascii should I remove just one character or whole word
# TODO: don't remove stopwords; at least for tf-idf it does not matter; for binary and BM25??

In [43]:
text_files=os.listdir('test-corpora')

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sunild/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# using library
import nltk
with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    tokens=nltk.word_tokenize(file)
    print(len(tokens))

13980


In [10]:
text=nltk.Text(tokens)

In [40]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [52]:
porter=PorterStemmer()
lancaster=LancasterStemmer()
porter.stem('played')


'play'

In [55]:
from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    # token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)



In [56]:

with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    x=stemSentence(file)
    print(x)


titl : javascript text : from wikipedia , the free encyclopedia thi is the latest accept revis , review on 2 februari 2022 . jump to navig jump to search .mw-parser-output .hatnot { font-styl : ital } .mw-parser-output div.hatnot { padding-left:1.6em ; margin-bottom:0.5em } .mw-parser-output .hatnot i { font-styl : normal } .mw-parser-output .hatnote+link+.hatnot { margin-top : -0.5em } not to be confus with java ( program languag ) or javanes script . `` .j '' redirect here . for the microsoft dialect use in internet explor , see jscript . for the use of javascript on wikipedia , see wikipedia : javascript . high-level program languag .mw-parser-output .infobox-subbox { padding:0 ; border : none ; margin : -3px ; width : auto ; min-width:100 % ; font-size:100 % ; clear : none ; float : none ; background-color : transpar } .mw-parser-output .infobox-3cols-child { margin : auto } .mw-parser-output .infobox .navbar { font-size:100 % } body.skin-minerva .mw-parser-output .infobox-head , b

In [18]:
text.concordance('javascript')

Displaying 25 of 178 matches:
Title : JavaScript Text : From Wikipedia , the free 
r , see JScript . For the uses of JavaScript on Wikipedia , see Wikipedia : Ja
pt on Wikipedia , see Wikipedia : JavaScript . High-level programming language
 , Objective-J , Opa , TypeScript JavaScript at Wikibooks JavaScript ( /ËdÊÉ
ypeScript JavaScript at Wikibooks JavaScript ( /ËdÊÉËvÉskrÉªpt/ ) , [ 10 
 [ 11 ] Over 97 % of websites use JavaScript on the client side for web page b
jor web browsers have a dedicated JavaScript engine to execute the code on use
ute the code on users ' devices . JavaScript is a high-level , often just-in-t
 or other runtime system provides JavaScript APIs for I/O . JavaScript engines
rovides JavaScript APIs for I/O . JavaScript engines were originally used only
ge is Node.js . Although Java and JavaScript are similar in name , syntax , an
er 1995 , the name was changed to JavaScript for the official release in Decem
 ] [ 1 ] [ 18 ] The choice of the JavaScript nam

In [5]:
with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    regex_exp3=re.compile(r'\n') # to sub \n by ' '
    regex_exp4=re.compile(r'\t') # to sub \n by ''
    clean_text=regex_exp4.sub('',regex_exp3.sub(' ',file))
    tokens=clean_text.split(' ')
    print(len(tokens))

NameError: name 're' is not defined

In [75]:
# using whitespace
with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    tokens=file.split(' ')
    print(len(tokens))

9314


In [68]:
with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    for c in file:
        if not c.isascii():
            file=file.replace(c,'')
            # print(c)
    print(file)


Title:
JavaScript
Text:

		From Wikipedia, the free encyclopedia
		
This is the latest accepted revision, reviewed on 2 February 2022.



		
		
		
		Jump to navigation
		Jump to search
		.mw-parser-output .hatnote{font-style:italic}.mw-parser-output div.hatnote{padding-left:1.6em;margin-bottom:0.5em}.mw-parser-output .hatnote i{font-style:normal}.mw-parser-output .hatnote+link+.hatnote{margin-top:-0.5em}Not to be confused with Java (programming language) or Javanese script.
".js" redirects here. For the Microsoft dialect used in Internet Explorer, see JScript.
For the uses of JavaScript on Wikipedia, see Wikipedia:JavaScript.


High-level programming language
.mw-parser-output .infobox-subbox{padding:0;border:none;margin:-3px;width:auto;min-width:100%;font-size:100%;clear:none;float:none;background-color:transparent}.mw-parser-output .infobox-3cols-child{margin:auto}.mw-parser-output .infobox .navbar{font-size:100%}body.skin-minerva .mw-parser-output .infobox-header,body.skin-minerva 

In [58]:
with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    # print(type(file))
    # using greedy approach
    # to handle css code
    regex_exp1=re.compile(r'.mw.*}')
    # to handle html tags
    regex_exp2=re.compile(r'<.*>')
    # can it handle all such CSS cases
    # but there are some html cases also
    # css_text=regex_exp.findall(file)
    clean_txt=regex_exp1.sub('',file)
    print(clean_txt)
    clean_txt=regex_exp2.sub('',clean_txt)


Title:
JavaScript
Text:

		From Wikipedia, the free encyclopedia
		
This is the latest accepted revision, reviewed on 2 February 2022.



		
		
		
		Jump to navigation
		Jump to search
		Not to be confused with Java (programming language) or Javanese script.
".js" redirects here. For the Microsoft dialect used in Internet Explorer, see JScript.
For the uses of JavaScript on Wikipedia, see Wikipedia:JavaScript.


High-level programming language
JavaScriptParadigmMulti-paradigm: event-driven, functional, imperative, object-oriented programmingDesignedÂ byBrendan Eich of Netscape initially; others have also contributed to the ECMAScript standardFirstÂ appearedDecemberÂ 4, 1995; 26 years agoÂ (1995-12-04)[1]Stable releaseECMAScript 2021[2]Â 
   / June 2021; 8 months agoÂ (June 2021)Preview releaseECMAScript 2022[3]Â 
   / 22 July 2021; 6 months agoÂ (22 July 2021)
Typing disciplineDynamic, weak, duckFilename extensions
.js
.cjs
.mjs[4]
Websitewww.ecma-international.org/publications-and-st

In [57]:
print(ascii('Â'))
print('Â'.isascii())
print('ACD'.isascii())

'\xc2'
False
True


In [59]:
print(clean_txt)


Title:
JavaScript
Text:

		From Wikipedia, the free encyclopedia
		
This is the latest accepted revision, reviewed on 2 February 2022.



		
		
		
		Jump to navigation
		Jump to search
		Not to be confused with Java (programming language) or Javanese script.
".js" redirects here. For the Microsoft dialect used in Internet Explorer, see JScript.
For the uses of JavaScript on Wikipedia, see Wikipedia:JavaScript.


High-level programming language
JavaScriptParadigmMulti-paradigm: event-driven, functional, imperative, object-oriented programmingDesignedÂ byBrendan Eich of Netscape initially; others have also contributed to the ECMAScript standardFirstÂ appearedDecemberÂ 4, 1995; 26 years agoÂ (1995-12-04)[1]Stable releaseECMAScript 2021[2]Â 
   / June 2021; 8 months agoÂ (June 2021)Preview releaseECMAScript 2022[3]Â 
   / 22 July 2021; 6 months agoÂ (22 July 2021)
Typing disciplineDynamic, weak, duckFilename extensions
.js
.cjs
.mjs[4]
Websitewww.ecma-international.org/publications-and-st